In [9]:
%reset -f

In [1]:
import tensorflow as tf
import os
import random
from keras_unet_collection import models, base, utils
import numpy as np
from tqdm import tqdm 
from keras import backend as K
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from skimage import io  # for reading TIFF images
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import *

import time

2024-10-04 14:13:28.018229: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-04 14:13:28.041198: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 14:13:28.041221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 14:13:28.041846: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 14:13:28.045924: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Set the desired number of patches
desired_num_patches = 1681

# Your data directories
optic_images_dir = 'Mosaic30m_patches_128px'
sar_images_dir = 'SAR_patches_128px'
agb_maps_dir = 'AGB128Pxpatches'
fvs_images_dir = 'FVS_Patches'
slope_images_dir = 'UNET_Slope_Patches/Patches'
aspect_images_dir = 'UNET_Aspect_Patches'
# Get the list of file names in the directories without sorting
agb_map_files = os.listdir(agb_maps_dir)
optic_image_files = os.listdir(optic_images_dir)
sar_image_files = os.listdir(sar_images_dir)
fvs_image_files = os.listdir(fvs_images_dir)
slope_image_files = os.listdir(slope_images_dir)
aspect_image_files = os.listdir(aspect_images_dir)

In [3]:


# Initialize empty lists to store data
x_train_list = []
y_train_list = []
agb_values_list = []

# Loop through each patch with tqdm for progress tracking
for i in tqdm(range(min(desired_num_patches, len(optic_image_files))), desc='Loading Patches'):
    # Get file names for the current index
    optic_file = optic_image_files[i]
    sar_file = sar_image_files[i]
    agb_file = agb_map_files[i]
    fvs_file = fvs_image_files[i]
    slope_file = slope_image_files[i]  # New Slope file
    aspect_file = aspect_image_files[i]  # New Aspect file
    
    # Load images
    optic_image = io.imread(os.path.join(optic_images_dir, optic_file))
    sar_image = io.imread(os.path.join(sar_images_dir, sar_file))
    agb_map = io.imread(os.path.join(agb_maps_dir, agb_file))
    fvs_image = io.imread(os.path.join(fvs_images_dir, fvs_file))  # Load FVS image
    slope_image = io.imread(os.path.join(slope_images_dir, slope_file))  # Load Slope image
    aspect_image = io.imread(os.path.join(aspect_images_dir, aspect_file))  # Load Aspect image

    # Ensure the optic image has 13 bands
    if optic_image.shape[-1] != 13:
        raise ValueError(f"Optic image should have 13 bands. Found: {optic_image.shape[-1]}")
    
    # Ensure FVS image has 7 bands
    if fvs_image.shape[-1] != 7:
        raise ValueError(f"FVS image should have 7 bands. Found: {fvs_image.shape[-1]}")
    
    # Ensure Slope and Aspect images are 2D (single band)
    if slope_image.ndim != 2 or aspect_image.ndim != 2:
        raise ValueError("Slope and Aspect images should be 2D.")
    
    # Normalize all datasets using min-max scaling
    optic_image_normalized = (optic_image - np.min(optic_image)) / (np.max(optic_image) - np.min(optic_image))
    sar_image_normalized = (sar_image - np.min(sar_image)) / (np.max(sar_image) - np.min(sar_image))
    fvs_image_normalized = (fvs_image - np.min(fvs_image)) / (np.max(fvs_image) - np.min(fvs_image))
    slope_image_normalized = (slope_image - np.min(slope_image)) / (np.max(slope_image) - np.min(slope_image))
    aspect_image_normalized = (aspect_image - np.min(aspect_image)) / (np.max(aspect_image) - np.min(aspect_image))

    # Expand the dimensions of Slope and Aspect to match the channel structure
    slope_image_expanded = np.expand_dims(slope_image_normalized, axis=-1)
    aspect_image_expanded = np.expand_dims(aspect_image_normalized, axis=-1)

    # Concatenate optic, SAR, FVS, Slope, and Aspect images along the channel axis
    x_train = np.concatenate([optic_image_normalized, sar_image_normalized, fvs_image_normalized, slope_image_expanded, aspect_image_expanded], axis=-1)

    # Store the concatenated data
    x_train_list.append(x_train)
    agb_values_list.append(agb_map)

    # Break the loop if the desired number of patches is reached
    if len(x_train_list) >= desired_num_patches:
        break

# Combine all patches into single arrays
x_data = np.stack(x_train_list)
agb_values = np.stack(agb_values_list)

# Normalize AGB values using min-max scaling
min_agb = np.min(agb_values)
max_agb = np.max(agb_values)
agb_scaled = (agb_values - min_agb) / (max_agb - min_agb)

# Reshape scaled AGB values for compatibility with the model
agb_normalized = np.expand_dims(agb_scaled, axis=-1)

# Split the data into training, validation, and test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, agb_normalized, test_size=0.20, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

# Print the shapes of the datasets to ensure they have the correct dimensions
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_val:', x_val.shape)
print('y_val:', y_val.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

input_shape = x_train.shape[1:]

Loading Patches: 100%|█████████▉| 1680/1681 [00:04<00:00, 365.70it/s]


x_train: (1075, 128, 128, 24)
y_train: (1075, 128, 128, 1)
x_val: (269, 128, 128, 24)
y_val: (269, 128, 128, 1)
x_test: (337, 128, 128, 24)
y_test: (337, 128, 128, 1)


In [ ]:

model = models.u2net_2d((128, 128, 24), n_labels=1, 
                        filter_num_down=[64, 128, 256, 512],
                        activation='ReLU', output_activation='Sigmoid', 
                        batch_norm=True, pool=False, unpool=False, deep_supervision=True, name='u2net')

In [ ]:
model = models.transunet_2d((512, 512, 3), filter_num=[64, 128, 256, 512], n_labels=1, stack_num_down=2, stack_num_up=2,
                                embed_dim=128, num_mlp=2048, num_heads=12, num_transformer=12,
                                activation='ReLU', mlp_activation='GELU', output_activation='Sigmoid', 
                                batch_norm=True, pool=True, unpool='bilinear', name='transunet')

In [ ]:
model.summary()

In [6]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-5
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [7]:
lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=10,
                               min_lr=0.5e-6)

In [8]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
#Modelcheckpoint
checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_TransUNET.h5',  monitor='val_mae' ,verbose=1, save_best_only=True)

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_mae'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        checkpointer, lr_reducer, lr_scheduler]



start_time = time.time()

results = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=4, epochs=120, callbacks=callbacks)

end_time = time.time()
training_time_seconds = end_time - start_time
training_time_minutes = training_time_seconds / 60

print("Training completed in {:.2f} seconds ({:.2f} minutes)".format(training_time_seconds, training_time_minutes))